In [ ]:
import re
import json
import sqlite3
import time
import random
from datetime import datetime
from pathlib import Path

import requests
from bs4 import BeautifulSoup

#設定
BASE_URL = "https://www.baitoru.com/kanto/jlist/yamanotesen/stp4/btp1/"
DB_PATH = Path("baitoru_jobs.db")
TARGET_COUNT = 1000

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}

TARGET_TAGS = {
    "週1〜OK",
    "日払い",
    "未経験OK",
    "学生",
    "シフト相談",
    "～4h/日",
    "～6h/日",
}


#sleep 設定
SLEEP_AFTER_PAGE = (1.5, 3.0)
SLEEP_PER_JOB = (0.2, 0.5)
SLEEP_BETWEEN_PAGES = (2.0, 4.0)

def rsleep(rng):
    time.sleep(random.uniform(*rng))

#DB
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

saved_count = 0
seen_job_ids = set()
page = 1

#ページループ
while saved_count < TARGET_COUNT:
    url = BASE_URL if page == 1 else f"{BASE_URL}page{page}/"
    print(f"\nPage {page}: {url}")

    res = requests.get(url, headers=HEADERS, timeout=30)
    if res.status_code != 200:
        print("ページ取得失敗")
        break

    soup = BeautifulSoup(res.text, "html.parser")
    rsleep(SLEEP_AFTER_PAGE)

    #求人カード単位で取得
    cards = soup.select('a[href*="/job"]')
    print(f"候補リンク: {len(cards)}")

    page_saved = 0

    for a in cards:
        if saved_count >= TARGET_COUNT:
            break

        href = a.get("href", "")
        m = re.search(r"job(\d+)", href)
        if not m:
            continue

        job_id = m.group(1)
        if job_id in seen_job_ids:
            continue
        seen_job_ids.add(job_id)

        #PR除外
        card = a.find_parent("article") or a.find_parent("div")
        if not card:
            continue
        if card.find("img", alt="PR"):
            continue

        card_text = card.get_text(" ", strip=True)

        #時給
        wages = re.findall(r"時給\s*([\d,]+)", card_text)
        if not wages:
            continue

        min_hourly_wage = min(int(w.replace(",", "")) for w in wages)

        #駅
        station = None
        m_station = re.search(r"([一-龥ぁ-んァ-ン]+駅)", card_text)
        if m_station:
            station = m_station.group(1)

        #タグ
        tags = [t for t in TARGET_TAGS if t in card_text]
        tags_json = json.dumps(tags, ensure_ascii=False)

        # DB UPSERT
        cur.execute("""
        INSERT INTO jobs (
            job_id,
            min_hourly_wage,
            nearest_station,
            tags,
            scraped_at
        ) VALUES (?, ?, ?, ?, ?)
        ON CONFLICT(job_id) DO UPDATE SET
            min_hourly_wage = excluded.min_hourly_wage,
            nearest_station = excluded.nearest_station,
            tags = excluded.tags,
            scraped_at = excluded.scraped_at
        """, (
            job_id,
            min_hourly_wage,
            station,
            tags_json,
            datetime.now().isoformat()
        ))

        saved_count += 1
        page_saved += 1

        print(
            f"{saved_count}/{TARGET_COUNT} "
            f"id={job_id} 時給={min_hourly_wage} 駅={station}"
        )

        rsleep(SLEEP_PER_JOB)

    conn.commit()
    print(f"Page {page} 保存: {page_saved} 件")

    if page_saved == 0:
        print("これ以上取得できなそうなので終了")
        break

    page += 1
    rsleep(SLEEP_BETWEEN_PAGES)

conn.close()
print(f"\n 完了")



Page 1: https://www.baitoru.com/kanto/jlist/yamanotesen/stp4/btp1/
候補リンク: 47
1/1000 id=155540052 時給=5000 駅=上野駅
2/1000 id=155540060 時給=4500 駅=上野駅
3/1000 id=155597979 時給=5000 駅=新橋駅
4/1000 id=154329085 時給=1600 駅=品川駅
5/1000 id=104205022 時給=1714 駅=新宿駅
6/1000 id=104205023 時給=1714 駅=池袋駅
7/1000 id=155237003 時給=1720 駅=上野駅
8/1000 id=155188302 時給=1250 駅=新宿駅
9/1000 id=143246693 時給=5000 駅=池袋駅
10/1000 id=148701846 時給=1800 駅=品川駅
11/1000 id=151782625 時給=2000 駅=新宿駅
12/1000 id=139523865 時給=1350 駅=上野駅
13/1000 id=154707510 時給=1400 駅=御徒町駅
14/1000 id=154999527 時給=1380 駅=原宿駅
15/1000 id=154999522 時給=1380 駅=駒込駅
16/1000 id=140499119 時給=1500 駅=池袋駅
17/1000 id=142647854 時給=1714 駅=上野駅
18/1000 id=106121162 時給=1714 駅=渋谷駅
19/1000 id=106121174 時給=1714 駅=秋葉原駅
20/1000 id=106121161 時給=1714 駅=上野駅
📦 Page 1 保存: 20 件

Page 2: https://www.baitoru.com/kanto/jlist/yamanotesen/stp4/btp1/page2/
候補リンク: 47
21/1000 id=154957536 時給=2500 駅=渋谷駅
22/1000 id=155562703 時給=1500 駅=トウェイ駅
23/1000 id=32897932 時給=1250 駅=渋谷駅
24/1000 id=143731387 